In [20]:
import pandas as pd # provides fast data analysis structures and tools
import numpy as np # faster way of working with arrays


import math
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import seaborn as sns # to visualize data
import matplotlib.pyplot as plt
import ipywidgets as widgets


import seaborn as sns # to visualize data
import matplotlib.pyplot as plt
import ipywidgets as widgets